In [1]:
import scipy.optimize
import numpy as np
import tensorflow as tf
import keras
from keras import layers
np.random.seed(2727)

c:\Users\jigna\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class Network:
    def __init__(self, input_dim) -> None:
        self.input_dim = input_dim

    def build(self):
        nn = keras.Sequential(
        [
            layers.Input(shape=(self.input_dim,)),
            layers.Dense(32, activation="tanh", kernel_initializer='he_normal'),
            layers.Dense(32, activation="tanh", kernel_initializer='he_normal'),
            layers.Dense(1),
        ])
        return nn




In [ ]:
network = Network(2).build()

In [ ]:
# Network test
network.summary()

In [12]:

class HeatModel:

    """
    
    This class performes PINN-type algorithm to numerically solve 1D heat equation.

    """

    def __init__(self, input_dim, x_train, y_train) -> None:

        # Atribute model represents the nn that approximates the pde solution.        
        self.model = Network(input_dim).build()

        self.x_train = x_train
        self.y_train = y_train

        self.batch_size = len(x_train[1])

    def pde_loss(self, int_data, int_pred):
        with tf.GradientTape() as tape2:
            tape2.watch(int_data)
            with tf.GradientTape() as tape1:
                tape1.watch(int_data)
                u = self.model(int_data)
            u_z = tape1.batch_jacobian(u, int_data)
        u_zz = tape2.batch_jacobian(u_z, int_data)
        u_t = u_z[..., 0, 0]
        u_xx = u_zz[..., 0, 1, 1]
        return tf.keras.losses.mse(u_t-u_xx, int_pred)



    def total_loss(self, weights):
        self.set_weights(flat_weights=weights)
        # Compute losses
        pde = self.pde_loss(self.x_train[0], self.y_train[0])
        initial = tf.keras.losses.mse(self.model(self.x_train[1]), self.u0(self.x_train[0]))
        left = tf.keras.losses.mse(self.model(self.x_train[2]), np.zeros(self.batch_size))
        right = tf.keras.losses.mse(self.model(self.x_train[3]), np.zeros(self.batch_size))
        
        return pde + initial + left + right
    
    
    @tf.function
    def tf_evaluate(self, x, y):
        with tf.GradientTape() as tape:
            loss = tf.reduce_mean(tf.keras.losses.mse(self.residue(x), y))
        grads = tape.gradient(loss, self.model.trainable_variabels)
        return loss, grads
    

    def set_weights(self, flat_weights):
        """Set the updated weights.

        Args:
            weights (np.array): current weight.
        """
        shapes = [w.shape for w in self.model.get_weights()]
        # Cumulative sum to get the correct indexes.
        split_ids = np.cumsum([np.prod(shape) for shape in [0] + shapes])
        weights = [flat_weights[from_id:to_id].reshape(shape)
            for from_id, to_id, shape in zip(split_ids[:-1], split_ids[1:], shapes) ]
        self.model.set_weights(weights)

    def fit(self):
        # Flattened weights.
        initial_weight = np.concatenate([w.flatten() for w in self.model.get_weights()])
        scipy.optimize.fmin_l_bfgs_b(func=self.evaluate, x0=initial_weight)

In [13]:
num_train_samples = 10
x_f = 2
x_ini = 0
t_f = 0.2

# pde loss training sample
tx_eqn = np.random.rand(num_train_samples, 2)
tx_eqn[..., 0] = t_f*tx_eqn[..., 0]               
tx_eqn[..., 1] = (x_f-x_ini)*tx_eqn[..., 1] + x_ini
tx_eqn = tf.Variable(tx_eqn, trainable=False)
# initial condition training sample           
tx_ini = np.random.rand(num_train_samples, 2)
tx_ini[..., 0] = 0                               
tx_ini[..., 1] = (x_f-x_ini)*tx_ini[..., 1] + x_ini
# up boundary training sample       
tx_bnd_up = np.random.rand(num_train_samples, 2)
tx_bnd_up[..., 0] = t_f*tx_bnd_up[..., 0]               
tx_bnd_up[..., 1] = x_f  # x = -1 or +1
# lower boundary training sample
tx_bnd_down = np.random.rand(num_train_samples, 2)
tx_bnd_down[..., 0] = t_f*tx_bnd_down[..., 0]              
tx_bnd_down[..., 1] = x_ini  

x_train = [tx_eqn, tx_ini, tx_bnd_up, tx_bnd_down]

In [14]:
heat_model = HeatModel(2, x_train, x_train)

In [15]:
loss = heat_model.total_loss(heat_model.model.get_weights())

AttributeError: 'list' object has no attribute 'reshape'

In [15]:
tf.keras.losses.mse(pde_loss, np.zeros((10)))

<tf.Tensor: shape=(), dtype=float64, numpy=3.3012380470507674>

In [ ]:
u_z[...,0,0]

In [ ]:
u_zz

In [ ]:
u_zz[...,0,1,1]


In [ ]:
model(tf.constant([[0,0,0]]))

In [ ]:
model.summary()